In [1]:
from ortools.sat.python import cp_model
from itertools import product

In [2]:
p = 127

def restrict(model, expr, lb, ub):
    model.Add(lb <= expr)
    model.Add(expr <= ub)

def model_constant_addition(model, u, v):
    model.Add(v >= u)

def model_power_map(model, u, v, d):
    e = model.NewIntVar(0, d-1, "")
    model.Add(u + e*(p-1) == d*v)
    restrict(model, d*v-e*p, 0, p-1)

def model_round_function(model, u, v):
    w = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)]
    # sbox
    for i in range(4):
        for j in range(4):
            model_power_map(model, u[i][j], w[i][j], 5)
    # shift rows
    for i in range(4):
        w[i] = w[i][i:] + w[i][:i]

    # mix columns and constant/key addition (constant from sbox is shifted through)
    for i in range(4): # column
        model.Add(sum(x[i] for x in v) >= sum(x[i] for x in w))

def model_final_round(model, u, v):
    w = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)]
    # sbox and constant/key addition
    for i in range(4):
        for j in range(4):
            model_power_map(model, u[i][j], w[i][j], 5)
            model_constant_addition(model, w[i][j], v[i][j])

# degree estimation
Comparison with paper: $e^r$

#### conclusion:
degree doesn't immediately hit the maximal degree when it can't grow exponentially anymore.

In [3]:
# of whole function
for r in range(1, 8):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for i in range(r-1):
        model_round_function(model, us[i], us[i+1])
    model_final_round(model, us[-2], us[-1])
    
    # output constraint
    model.Add(sum(sum(x) for x in us[-1]) == 1)
    
    model.Maximize(sum(sum(x) for x in us[0]))
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL:
        print(r, f"maximal degree is: {solver.ObjectiveValue()}")
        if solver.ObjectiveValue() == 16*(p-1)-1:
            break
    elif status == cp_model.FEASIBLE:
        print(r, f"lower bound on the degree is: {solver.ObjectiveValue()}")
        break
    else:
        print(r, "error")
        break

1 maximal degree is: 5.0
2 maximal degree is: 25.0
3 maximal degree is: 125.0
4 maximal degree is: 625.0
5 maximal degree is: 2002.0
6 maximal degree is: 2012.0
7 maximal degree is: 2015.0


In [4]:
# of component functions
for r in range(1, 8):
    for i in range(4):
        for j in range(4):
            model = cp_model.CpModel()
            us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
            
            # model function
            for k in range(r-1):
                model_round_function(model, us[k], us[k+1])
            model_final_round(model, us[-2], us[-1])
            
            # output constraint
            for k in range(4):
                for l in range(4):
                    if k == i and l == j:
                        model.Add(us[-1][k][l] == 1)
                    else:
                        model.Add(us[-1][k][l] == 0)
            
            model.Maximize(sum(sum(x) for x in us[0]))
            solver = cp_model.CpSolver()
            solver.parameters.num_workers = 1
            status = solver.Solve(model)
            
            # depending on the status of the solver, print the results and a message
            if status == cp_model.OPTIMAL:
                print(r, i, j, f"maximal degree is: {solver.ObjectiveValue()}")
            elif status == cp_model.FEASIBLE:
                print(r, i, j, f"lower bound on the degree is: {solver.ObjectiveValue()}")
                break
            else:
                print(r, i, j, "error")
                break

1 0 0 maximal degree is: 5.0
1 0 1 maximal degree is: 5.0
1 0 2 maximal degree is: 5.0
1 0 3 maximal degree is: 5.0
1 1 0 maximal degree is: 5.0
1 1 1 maximal degree is: 5.0
1 1 2 maximal degree is: 5.0
1 1 3 maximal degree is: 5.0
1 2 0 maximal degree is: 5.0
1 2 1 maximal degree is: 5.0
1 2 2 maximal degree is: 5.0
1 2 3 maximal degree is: 5.0
1 3 0 maximal degree is: 5.0
1 3 1 maximal degree is: 5.0
1 3 2 maximal degree is: 5.0
1 3 3 maximal degree is: 5.0
2 0 0 maximal degree is: 25.0
2 0 1 maximal degree is: 25.0
2 0 2 maximal degree is: 25.0
2 0 3 maximal degree is: 25.0
2 1 0 maximal degree is: 25.0
2 1 1 maximal degree is: 25.0
2 1 2 maximal degree is: 25.0
2 1 3 maximal degree is: 25.0
2 2 0 maximal degree is: 25.0
2 2 1 maximal degree is: 25.0
2 2 2 maximal degree is: 25.0
2 2 3 maximal degree is: 25.0
2 3 0 maximal degree is: 25.0
2 3 1 maximal degree is: 25.0
2 3 2 maximal degree is: 25.0
2 3 3 maximal degree is: 25.0
3 0 0 maximal degree is: 125.0
3 0 1 maximal degree is: 

# presence of all univariate monomials
According to the paper, after 4 rounds, all univariate monomials should be present. Is this in all component functions or in at least one component function?

#### conclusion:
5 rounds are necessary, the univariate degree does not go up from round 3 to round 4

In [5]:
r = 3
for a1, a2, b1, b2 in product(range(4), repeat=4):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r):
        model_round_function(model, us[k], us[k+1])
    
    # output constraint
    for i1, i2 in product(range(4), repeat=2):
        if i1 != a1 or i2 != a2:
            model.Add(us[-1][i1][i2] == 0)
        else:
            model.Add(us[-1][i1][i2] == 1)
    
    model.Maximize(us[0][a1][a2])
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL:
        print(a1, a2, b1, b2, f"maximal degree is: {solver.ObjectiveValue()}")
    elif status == cp_model.FEASIBLE:
        print(a1, a2, b1, b2, f"lower bound on the degree is: {solver.ObjectiveValue()}")
        break
    else:
        print(a1, a2, b1, b2, "error")
        break

0 0 0 0 maximal degree is: 125.0
0 0 0 1 maximal degree is: 125.0
0 0 0 2 maximal degree is: 125.0
0 0 0 3 maximal degree is: 125.0
0 0 1 0 maximal degree is: 125.0
0 0 1 1 maximal degree is: 125.0
0 0 1 2 maximal degree is: 125.0
0 0 1 3 maximal degree is: 125.0
0 0 2 0 maximal degree is: 125.0
0 0 2 1 maximal degree is: 125.0
0 0 2 2 maximal degree is: 125.0
0 0 2 3 maximal degree is: 125.0
0 0 3 0 maximal degree is: 125.0
0 0 3 1 maximal degree is: 125.0
0 0 3 2 maximal degree is: 125.0
0 0 3 3 maximal degree is: 125.0
0 1 0 0 maximal degree is: 125.0
0 1 0 1 maximal degree is: 125.0
0 1 0 2 maximal degree is: 125.0
0 1 0 3 maximal degree is: 125.0
0 1 1 0 maximal degree is: 125.0
0 1 1 1 maximal degree is: 125.0
0 1 1 2 maximal degree is: 125.0
0 1 1 3 maximal degree is: 125.0
0 1 2 0 maximal degree is: 125.0
0 1 2 1 maximal degree is: 125.0
0 1 2 2 maximal degree is: 125.0
0 1 2 3 maximal degree is: 125.0
0 1 3 0 maximal degree is: 125.0
0 1 3 1 maximal degree is: 125.0
0 1 3 2 ma

In [6]:
r = 4
for a1, a2, b1, b2 in product(range(4), repeat=4):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r):
        model_round_function(model, us[k], us[k+1])
    
    # output constraint
    for i1, i2 in product(range(4), repeat=2):
        if i1 != a1 or i2 != a2:
            model.Add(us[-1][i1][i2] == 0)
        else:
            model.Add(us[-1][i1][i2] == 1)
    
    model.Maximize(us[0][a1][a2])
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL:
        print(a1, a2, b1, b2, f"maximal degree is: {solver.ObjectiveValue()}")
    elif status == cp_model.FEASIBLE:
        print(a1, a2, b1, b2, f"lower bound on the degree is: {solver.ObjectiveValue()}")
        break
    else:
        print(a1, a2, b1, b2, "error")
        break

0 0 0 0 maximal degree is: 125.0
0 0 0 1 maximal degree is: 125.0
0 0 0 2 maximal degree is: 125.0
0 0 0 3 maximal degree is: 125.0
0 0 1 0 maximal degree is: 125.0
0 0 1 1 maximal degree is: 125.0
0 0 1 2 maximal degree is: 125.0
0 0 1 3 maximal degree is: 125.0
0 0 2 0 maximal degree is: 125.0
0 0 2 1 maximal degree is: 125.0
0 0 2 2 maximal degree is: 125.0
0 0 2 3 maximal degree is: 125.0
0 0 3 0 maximal degree is: 125.0
0 0 3 1 maximal degree is: 125.0
0 0 3 2 maximal degree is: 125.0
0 0 3 3 maximal degree is: 125.0
0 1 0 0 maximal degree is: 125.0
0 1 0 1 maximal degree is: 125.0
0 1 0 2 maximal degree is: 125.0
0 1 0 3 maximal degree is: 125.0
0 1 1 0 maximal degree is: 125.0
0 1 1 1 maximal degree is: 125.0
0 1 1 2 maximal degree is: 125.0
0 1 1 3 maximal degree is: 125.0
0 1 2 0 maximal degree is: 125.0
0 1 2 1 maximal degree is: 125.0
0 1 2 2 maximal degree is: 125.0
0 1 2 3 maximal degree is: 125.0
0 1 3 0 maximal degree is: 125.0
0 1 3 1 maximal degree is: 125.0
0 1 3 2 ma

In [7]:
r = 5
for a1, a2, b1, b2 in product(range(4), repeat=4):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r):
        model_round_function(model, us[k], us[k+1])
    
    # output constraint
    for i1, i2 in product(range(4), repeat=2):
        if i1 != a1 or i2 != a2:
            model.Add(us[-1][i1][i2] == 0)
        else:
            model.Add(us[-1][i1][i2] == 1)
    
    model.Maximize(us[0][a1][a2])
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL:
        print(a1, a2, b1, b2, f"maximal degree is: {solver.ObjectiveValue()}")
    elif status == cp_model.FEASIBLE:
        print(a1, a2, b1, b2, f"lower bound on the degree is: {solver.ObjectiveValue()}")
        break
    else:
        print(a1, a2, b1, b2, "error")
        break

0 0 0 0 maximal degree is: 126.0
0 0 0 1 maximal degree is: 126.0
0 0 0 2 maximal degree is: 126.0
0 0 0 3 maximal degree is: 126.0
0 0 1 0 maximal degree is: 126.0
0 0 1 1 maximal degree is: 126.0
0 0 1 2 maximal degree is: 126.0
0 0 1 3 maximal degree is: 126.0
0 0 2 0 maximal degree is: 126.0
0 0 2 1 maximal degree is: 126.0
0 0 2 2 maximal degree is: 126.0
0 0 2 3 maximal degree is: 126.0
0 0 3 0 maximal degree is: 126.0
0 0 3 1 maximal degree is: 126.0
0 0 3 2 maximal degree is: 126.0
0 0 3 3 maximal degree is: 126.0
0 1 0 0 maximal degree is: 126.0
0 1 0 1 maximal degree is: 126.0
0 1 0 2 maximal degree is: 126.0
0 1 0 3 maximal degree is: 126.0
0 1 1 0 maximal degree is: 126.0
0 1 1 1 maximal degree is: 126.0
0 1 1 2 maximal degree is: 126.0
0 1 1 3 maximal degree is: 126.0
0 1 2 0 maximal degree is: 126.0
0 1 2 1 maximal degree is: 126.0
0 1 2 2 maximal degree is: 126.0
0 1 2 3 maximal degree is: 126.0
0 1 3 0 maximal degree is: 126.0
0 1 3 1 maximal degree is: 126.0
0 1 3 2 ma

# max-data properties
dense after 7 rounds instead of 10.

In [8]:
r = 7
for b1, b2 in product(range(4), repeat=2):
    for a1, a2 in product(range(4), repeat=2):
        model = cp_model.CpModel()
        us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
        
        # model function
        for k in range(r-1):
            model_round_function(model, us[k], us[k+1])
        model_final_round(model, us[-2], us[-1])
        
        # input/output constraint
        for k in range(4):
            for l in range(4):
                if k == a1 and l == a2:
                    model.Add(us[0][k][l] >= p-2)
                else:
                    model.Add(us[0][k][l] >= p-1)
                if k == b1 and l == b2:
                    model.Add(us[-1][k][l] == 1)
                else:
                    model.Add(us[-1][k][l] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 1
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a1, a2, b1, b2, "zero sum")
        else:
            print(a1, a2, b1, b2, "error")

# Minimum data property

In [9]:
# single word subgroup
r = 2
a1, a2 = 0, 0
for b1, b2 in product(range(4), repeat=2):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r-1):
        model_round_function(model, us[k], us[k+1])
    model_final_round(model, us[-2], us[-1])
    
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == a1 and l == a2:
                model.Add(us[0][k][l] >= 63)
            else:
                model.Add(us[0][k][l] >= 0)
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # print(a, b, "trail exists")
        # for u in us:
        #     print([solver.Value(x) for x in u])
        pass
    elif status == cp_model.INFEASIBLE:
        print(b1, b2, "zero sum")
    else:
        print(b1, b2, "error")

0 0 zero sum
0 1 zero sum
0 2 zero sum
0 3 zero sum
1 0 zero sum
1 1 zero sum
1 2 zero sum
1 3 zero sum
2 0 zero sum
2 1 zero sum
2 2 zero sum
2 3 zero sum
3 0 zero sum
3 1 zero sum
3 2 zero sum
3 3 zero sum


In [10]:
# single word saturated
r = 4
a1, a2 = 0, 0
for b1, b2 in product(range(4), repeat=2):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r-1):
        model_round_function(model, us[k], us[k+1])
    model_final_round(model, us[-2], us[-1])
    
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == a1 and l == a2:
                model.Add(us[0][k][l] >= p-1)
            else:
                model.Add(us[0][k][l] >= 0)
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # print(a, b, "trail exists")
        # for u in us:
        #     print([solver.Value(x) for x in u])
        pass
    elif status == cp_model.INFEASIBLE:
        print(b1, b2, "zero sum")
    else:
        print(b1, b2, "error")

0 0 zero sum
0 1 zero sum
0 2 zero sum
0 3 zero sum
1 0 zero sum
1 1 zero sum
1 2 zero sum
1 3 zero sum
2 0 zero sum
2 1 zero sum
2 2 zero sum
2 3 zero sum
3 0 zero sum
3 1 zero sum
3 2 zero sum
3 3 zero sum


In [11]:
# diagonal
r = 5
for b1, b2 in product(range(4), repeat=2):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r-1):
        model_round_function(model, us[k], us[k+1])
    model_final_round(model, us[-2], us[-1])
    
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == l:
                model.Add(us[0][k][l] >= p-1)
            else:
                model.Add(us[0][k][l] >= 0)
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # print(a, b, "trail exists")
        # for u in us:
        #     print([solver.Value(x) for x in u])
        pass
    elif status == cp_model.INFEASIBLE:
        print(b1, b2, "zero sum")
    else:
        print(b1, b2, "error")

0 0 zero sum
0 1 zero sum
0 2 zero sum
0 3 zero sum
1 0 zero sum
1 1 zero sum
1 2 zero sum
1 3 zero sum
2 0 zero sum
2 1 zero sum
2 2 zero sum
2 3 zero sum
3 0 zero sum
3 1 zero sum
3 2 zero sum
3 3 zero sum


In [12]:
# double diagonal
r = 6
for b1, b2 in product(range(4), repeat=2):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r-1):
        model_round_function(model, us[k], us[k+1])
    model_final_round(model, us[-2], us[-1])
    
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == l or k == (l+1) % 4:
                model.Add(us[0][k][l] >= p-1)
            else:
                model.Add(us[0][k][l] >= 0)
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # print(a, b, "trail exists")
        # for u in us:
        #     print([solver.Value(x) for x in u])
        pass
    elif status == cp_model.INFEASIBLE:
        print(b1, b2, "zero sum")
    else:
        print(b1, b2, "error")

0 0 zero sum
0 1 zero sum
0 2 zero sum
0 3 zero sum
1 0 zero sum
1 1 zero sum
1 2 zero sum
1 3 zero sum
2 0 zero sum
2 1 zero sum
2 2 zero sum
2 3 zero sum
3 0 zero sum
3 1 zero sum
3 2 zero sum
3 3 zero sum


In [13]:
# diagonal + 1 saturated word
r = 6
for b1, b2 in product(range(4), repeat=2):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
    
    # model function
    for k in range(r-1):
        model_round_function(model, us[k], us[k+1])
    model_final_round(model, us[-2], us[-1])
    
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == l:
                model.Add(us[0][k][l] >= p-1)
            else:
                model.Add(us[0][k][l] >= 0)
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    model.Add(us[0][1][0] >= 63)
    model.Add(us[0][2][1] >= p-1)
    model.Add(us[0][3][2] >= p-1)
    model.Add(us[0][0][3] >= p-1)
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # print(a, b, "trail exists")
        # for u in us:
        #     print([solver.Value(x) for x in u])
        pass
    elif status == cp_model.INFEASIBLE:
        print(b1, b2, "zero sum")
    else:
        print(b1, b2, "error")

# higher divisibility
We can only find higher divisibility properties up to 3.5 rounds, up to $p^4$ for $p^{15}$ data and $p^3$ for $p^{12}$ data an $p^2$ for $p^8$ data

In [14]:
p = 127

def model_addition_divisibility(model, u0, u1, v):
    e = model.NewBoolVar("")
    model.Add(v == u0 + u1 + e*(1-p))
    return e

def model_constant_addition_divisibility(model, u, v, s):
    e = model.NewBoolVar("")
    model.add(v-u + e*(p-1) >= 0)
    model.add(v-u + e*(p-1) <= p-2)
    model.add(e <= (1-s)) # if saturated (s=1), just propagate u=v=p-1
    return e

def model_round_function_divisibility(model, u, v, sin, sout):
    w = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)]
    sin = list(sin)
    cor = 0
    # sbox
    for i in range(4):
        for j in range(4):
            model_power_map(model, u[i][j], w[i][j], 5)
    # shift rows
    for i in range(4):
        w[i] = w[i][i:] + w[i][:i]
        sin[i] = sin[i][i:] + sin[i][:i]

    # mix columns and constant/key addition (constant from sbox is shifted through)
    assignments_saturation = [(4, 4, 0)] + sum([[(i, 0, j) for j in range(5)] for i in range(4)], []) 
    for i in range(4): # column
        t1, t2, d = model.NewIntVar(0, 4, ""), model.NewIntVar(0, 4, ""), model.NewIntVar(0, 4, "")
        model.Add(t1 == sum(x[i] for x in sin))
        model.Add(t2 == sum(x[i] for x in sout))
        model.AddAllowedAssignments([t1, t2, d], assignments_saturation)
        model.Add((p-1)*d >= sum(x[i] for x in w) - sum(x[i] for x in v))
        cor += d
    return cor

def model_final_round_divisibility(model, u, v, sin):
    w = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)]
    cor = 0
    # sbox and constant/key addition
    for i in range(4):
        for j in range(4):
            model_power_map(model, u[i][j], w[i][j], 5)
            cor += model_constant_addition_divisibility(model, w[i][j], v[i][j], sin[i][j])
    return cor

In [15]:
# degree growth mod p^2
b1, b2 = 0, 0
for r in range(1, 8):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
    saturation =  [[[0]*4]*4]+ [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
    cor = 0
    
    # model function
    for i in range(4):
        for j in range(4):
            cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
    for k in range(r):
        cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
    # cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    model.Add(cor <= 1)
    model.Maximize(sum(sum(x) for x in us[0]))
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(int(solver.ObjectiveValue()))
        if solver.ObjectiveValue() == 126*16:
            break

505
2000
2012
2015
2015
2015
2016


In [16]:
# degree growth mod p^3
b1, b2 = 0, 0
for r in range(1, 8):
    model = cp_model.CpModel()
    us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
    saturation =  [[[0]*4]*4]+ [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
    cor = 0
    
    # model function
    for i in range(4):
        for j in range(4):
            cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
    for k in range(r):
        cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
    # cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
    # input/output constraint
    for k in range(4):
        for l in range(4):
            if k == b1 and l == b2:
                model.Add(us[-1][k][l] == 1)
            else:
                model.Add(us[-1][k][l] == 0)
    model.Add(cor <= 2)
    model.Maximize(sum(sum(x) for x in us[0]))
    solver = cp_model.CpSolver()
    solver.parameters.num_workers = 1
    status = solver.Solve(model)
    
    # depending on the status of the solver, print the results and a message
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(int(solver.ObjectiveValue()))
        if solver.ObjectiveValue() == 126*16:
            break

1005
2004
2013
2016


In [17]:
# 3.5 rounds
r = 4
a1, a2 = 0, 0
b1, b2 = 0, 0
model = cp_model.CpModel()
us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
saturation = [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
cor = 0

# model function
for i in range(4):
    for j in range(4):
        cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
for k in range(r-1):
    cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
# input/output constraint
for k in range(4):
    for l in range(4):
        if k == a1 and l == a2:
            model.Add(us[0][k][l] >= 0)
            model.Add(saturation[0][k][l] == 0)
        else:
            model.Add(us[0][k][l] >= p-1)
            model.Add(saturation[0][k][l] == 1)
        if k == b1 and l == b2:
            model.Add(us[-1][k][l] == 1)
        else:
            model.Add(us[-1][k][l] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 1
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("zero sum")
else:
    print("error")

divisibility by p^4.0


In [18]:
# 4 rounds
r = 4
a1, a2 = 0, 0
b1, b2 = 0, 0
model = cp_model.CpModel()
us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
saturation = [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r+1)]
cor = 0

# model function
for i in range(4):
    for j in range(4):
        cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
for k in range(r):
    cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
# input/output constraint
for k in range(4):
    for l in range(4):
        if k == a1 and l == a2:
            model.Add(us[0][k][l] >= 0)
            model.Add(saturation[0][k][l] == 0)
        else:
            model.Add(us[0][k][l] >= p-1)
            model.Add(saturation[0][k][l] == 1)
        if k == b1 and l == b2:
            model.Add(us[-1][k][l] == 1)
        else:
            model.Add(us[-1][k][l] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 1
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("zero sum")
else:
    print("error")

divisibility by p^1.0


In [19]:
# 3.5 rounds
r = 4
b1, b2 = 0, 0
model = cp_model.CpModel()
us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
saturation = [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
cor = 0

# model function
for i in range(4):
    for j in range(4):
        cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
for k in range(r-1):
    cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
# input/output constraint
for k in range(4):
    for l in range(4):
        if k == l:
            model.Add(us[0][k][l] >= 0)
            model.Add(saturation[0][k][l] == 0)
        else:
            model.Add(us[0][k][l] >= p-1)
            model.Add(saturation[0][k][l] == 1)
        if k == b1 and l == b2:
            model.Add(us[-1][k][l] == 1)
        else:
            model.Add(us[-1][k][l] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 1
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("zero sum")
else:
    print("error")

divisibility by p^3.0


In [20]:
# 3.5 rounds
r = 4
b1, b2 = 0, 0
model = cp_model.CpModel()
us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
saturation = [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
cor = 0

# model function
for i in range(4):
    for j in range(4):
        cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
for k in range(r-1):
    cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
# input/output constraint
for k in range(4):
    for l in range(4):
        if k == l or k == (l + 1)%4:
            model.Add(us[0][k][l] >= 0)
            model.Add(saturation[0][k][l] == 0)
        else:
            model.Add(us[0][k][l] >= p-1)
            model.Add(saturation[0][k][l] == 1)
        if k == b1 and l == b2:
            model.Add(us[-1][k][l] == 1)
        else:
            model.Add(us[-1][k][l] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 1
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("zero sum")
else:
    print("error")

divisibility by p^2.0


In [21]:
# 3.5 rounds
r = 4
b1, b2 = 0, 0
model = cp_model.CpModel()
us = [[[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] for _ in range(r+2)]
saturation = [[[model.NewBoolVar("") for _ in range(4)] for _ in range(4)] for _ in range(r)]
cor = 0

# model function
for i in range(4):
    for j in range(4):
        cor += model_constant_addition_divisibility(model, us[0][i][j],  us[1][i][j], saturation[0][i][j])
for k in range(r-1):
    cor += model_round_function_divisibility(model, us[k+1], us[k+2], saturation[k], saturation[k+1])
cor += model_final_round_divisibility(model, us[-2], us[-1], saturation[-1])
# input/output constraint
for k in range(4):
    for l in range(4):
        if k == l:
            model.Add(us[0][k][l] >= p-1)
            model.Add(saturation[0][k][l] == 1)
        else:
            model.Add(us[0][k][l] >= 0)
            model.Add(saturation[0][k][l] == 0)
        if k == b1 and l == b2:
            model.Add(us[-1][k][l] == 1)
        else:
            model.Add(us[-1][k][l] == 0)
model.Minimize(cor)
solver = cp_model.CpSolver()
solver.parameters.num_workers = 1
status = solver.Solve(model)

# depending on the status of the solver, print the results and a message
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"divisibility by p^{solver.ObjectiveValue()}")
elif status == cp_model.INFEASIBLE:
    print("zero sum")
else:
    print("error")

divisibility by p^1.0


# implementation and tests

In [22]:
from galois import GF
import numpy as np
F = GF(127)
M = F([[1, 1, 1, 1], [1, 2, 4, 16], [1, 4, 16, 2], [1, 16, 2, 4]])
c = F([[2]*4]*4)
def eval(x, k):
    r = len(k)-1
    for i in range(r-1):
        x = np.power(x + k[i], 5) + c
        for i in range(1, 4):
            x[i, :4-i], x[i, 4-i:] = x[i, i:], x[i, :i]
        x = M@x
        
    return np.power(x + k[r-1], 5) + k[r]

In [23]:
r = 4
k = [F.Random((4, 4)) for _ in range(r+1)]
acc = F.Zeros((4, 4))
for i in range(127):
    x = F.Zeros((4, 4))
    x[0, 0] = i
    acc += eval(x, k)
print(acc)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
